In [99]:
import pandas as pd
import datetime

In [100]:
ques_all_df = pd.read_csv('./datas/canact_all.csv')
ques_all_df['activity_group'].unique()
ques_all_df.columns

Index(['Unnamed: 0', 'local_id', 'activity_type_id', 'activity',
       'activity_group', 'created_at', 'started_at', 'finished_at',
       'deleted_at', 'user_id', 'user', 'user_tag', 'target_user_ids',
       'customer_id', 'customer', 'floor', 'record_type_id', 'name', 'value'],
      dtype='object')

In [101]:
# ques_all_df.query('activity_group=="業務関係"')
ques_all_df = ques_all_df[['activity_type_id','activity','activity_group','started_at','finished_at','deleted_at','user_id','record_type_id','name','value']]
# ques_all_df = ques_all_df.rename(columns={'finished_at':'datetime'})

In [102]:
drop_index = ques_all_df.index[ques_all_df['deleted_at'].notna()]
ques_all_df = ques_all_df.drop(drop_index)
# ques_all_df.to_csv('./datas/canact_all_preprocessed.csv')

In [103]:
ques_all_df['finished_at'] = pd.to_datetime(ques_all_df['finished_at'])
ques_all_df['started_at'] = pd.to_datetime(ques_all_df['started_at'])
ques_all_df['time_spent'] = ques_all_df['finished_at']-ques_all_df['started_at']
ques_work = ques_all_df.query('activity_group=="業務関係"')
ques_notwork = ques_all_df.query('activity_group=="業務外関係"')
ques_action = ques_all_df.query('activity_group=="業務外関係" or activity_group=="業務関係"')

In [104]:
ques_action['activity'].unique()

array(['食事（業務時間外）', '単独作業', '趣味・休憩', '家事・育児', '休憩（業務時間内）', '移動', 'Web会議',
       'その他', '業務外作業', '対面会議', '共同作業（コミュニケーションを伴う）', '電話（会議）'],
      dtype=object)

In [134]:
ques_action['meal'] = 0
ques_action['working_alone'] = 0
ques_action['hobby_break'] = 0
ques_action['housework_childcare'] = 0
ques_action['break'] = 0
ques_action['transfer'] = 0
ques_action['web_conference'] = 0
ques_action['other'] = 0
ques_action['work_outsideduty'] = 0
ques_action['facetoface_meeting'] = 0
ques_action['collaboration'] = 0
ques_action['remote_conference'] = 0

ques_action.loc[ques_action['activity']=='食事（業務時間外）', 'meal'] = ques_action['time_spent']
ques_action.loc[ques_action['activity']=='単独作業', 'working_alone'] = ques_action['time_spent']
ques_action.loc[ques_action['activity']=='趣味・休憩', 'hobby_break'] = ques_action['time_spent']
ques_action.loc[ques_action['activity']=='家事・育児', 'housework_childcare'] = ques_action['time_spent']
ques_action.loc[ques_action['activity']=='休憩（業務時間内）', 'break'] = ques_action['time_spent']
ques_action.loc[ques_action['activity']=='移動', 'transfer'] = ques_action['time_spent']
ques_action.loc[ques_action['activity']=='Web会議', 'web_conference'] = ques_action['time_spent']
ques_action.loc[ques_action['activity']=='その他', 'other'] = ques_action['time_spent']
ques_action.loc[ques_action['activity']=='業務外作業', 'work_outsideduty'] = ques_action['time_spent']
ques_action.loc[ques_action['activity']=='対面会議', 'facetoface_meeting'] = ques_action['time_spent']
ques_action.loc[ques_action['activity']=='共同作業（コミュニケーションを伴う）', 'collaboration'] = ques_action['time_spent']
ques_action.loc[ques_action['activity']=='電話（会議）', 'remote_conference'] = ques_action['time_spent']

In [135]:
import datetime as dt
ques_action['Date'] = ques_action['finished_at'].dt.date
ques_action['Time'] = ques_action['finished_at'].dt.hour
ques_action['am_pm'] = 0
ques_action.loc[(ques_action['Time']>11 )&(ques_action['Time']<=23), 'am_pm'] = 1

In [136]:
ques_action = ques_action.dropna(subset=['finished_at'])
ques_action = ques_action

In [137]:
ques_action = ques_action[['activity_type_id', 'activity', 'activity_group','Date', 'Time', 'am_pm', 'started_at',
       'finished_at','time_spent','user_id', 'record_type_id', 'name',
       'value', 'meal', 'working_alone', 'hobby_break',
       'housework_childcare', 'break', 'transfer', 'web_conference', 'other',
       'work_outsideduty', 'facetoface_meeting', 'collaboration',
       'remote_conference']]
ques_action

,activity_type_id,activity,activity_group,Date,Time,am_pm,started_at,finished_at,time_spent,user_id,...,hobby_break,housework_childcare,break,transfer,web_conference,other,work_outsideduty,facetoface_meeting,collaboration,remote_conference
26,10.0,食事（業務時間外）,業務外関係,2021-01-11,1,0,2021-01-11 00:25:01.106000+00:00,2021-01-11 01:37:14.232000+00:00,0 days 01:12:13.126000,93,...,0,0,0,0,0,0,0,0,0,0
27,5.0,単独作業,業務関係,2021-01-11,0,0,2021-01-11 00:54:06.356000+00:00,2021-01-11 00:54:09.060000+00:00,0 days 00:00:02.704000,7,...,0,0,0,0,0,0,0,0,0,0
28,8.0,趣味・休憩,業務外関係,2021-01-11,5,0,2021-01-11 00:59:35.453000+00:00,2021-01-11 05:41:05.205000+00:00,0 days 04:41:29.752000,30,...,0 days 04:41:29.752000,0,0,0,0,0,0,0,0,0
31,9.0,家事・育児,業務外関係,2021-01-11,3,0,2021-01-11 01:38:55.700000+00:00,2021-01-11 03:35:01.069000+00:00,0 days 01:56:05.369000,93,...,0,0 days 01:56:05.369000,0,0,0,0,0,0,0,0
32,10.0,食事（業務時間外）,業務外関係,2021-01-11,2,0,2021-01-11 01:47:12.908000+00:00,2021-01-11 02:58:14.532000+00:00,0 days 01:11:01.624000,48,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172246,10.0,食事（業務時間外）,業務外関係,2021-01-31,9,0,2021-01-31 09:08:36.930000+00:00,2021-01-31 09:08:38.819000+00:00,0 days 00:00:01.889000,82,...,0,0,0,0,0,0,0,0,0,0
172247,10.0,食事（業務時間外）,業務外関係,2021-02-01,9,0,2021-01-31 11:09:00+00:00,2021-02-01 09:09:00+00:00,0 days 22:00:00,82,...,0,0,0,0,0,0,0,0,0,0
172252,11.0,移動,業務外関係,2021-01-31,22,1,2021-01-31 21:39:01.968000+00:00,2021-01-31 22:37:43.159000+00:00,0 days 00:58:41.191000,82,...,0,0,0,0 days 00:58:41.191000,0,0,0,0,0,0
172254,10.0,食事（業務時間外）,業務外関係,2021-01-31,21,1,2021-01-31 21:45:35.629000+00:00,2021-01-31 21:45:37.518000+00:00,0 days 00:00:01.889000,82,...,0,0,0,0,0,0,0,0,0,0
